In [1]:
%load_ext sql

In [2]:
import psycopg2
import os
import re
from sqlalchemy import create_engine
from sqlalchemy import inspect
import pandas as pd
from langchain_community.chat_models import ChatOllama
from langchain.chains import create_sql_query_chain
from langchain_community.utilities import SQLDatabase 
from langchain_experimental.sql import SQLDatabaseChain

In [3]:
host = "localhost"
database = "LLama"
user = os.getenv('SQL_USER')
password = os.getenv('SQL_PASSWORD')

In [4]:
connection_string = f"postgresql://{user}:{password}@{host}/{database}"
engine = create_engine(connection_string)
insp = inspect(engine)
insp.get_table_names()

['transcript2', 'transcript', 'trial', 'trial2', 'trial 3', 'essays']

In [5]:
!ollama list

NAME                       ID              SIZE      MODIFIED   
nomic-embed-text:latest    0a109f422b47    274 MB    3 days ago    
llama2:latest              78e26419b446    3.8 GB    3 days ago    
gemma:7b                   a72c7f4d0a15    5.0 GB    3 days ago    
mistral:latest             f974a74358d6    4.1 GB    3 days ago    


## Dataset 1: Transcript

In [6]:
#loading the dataset from the DB
df = pd.read_sql('SELECT content from transcript;', engine)

# Retrieve the text data for index 0
text = df.loc[0, 'content']
text

"some kind of a crazy quantum mechanical system that somehow gives you buffer overflow, somehow\ngives you a rounding error in the floating point.\nSynthetic intelligences are kind of like the next stage of development.\nAnd I don't know where it leads to.\nLike at some point, I suspect the universe is some kind of a puzzle.\nThese synthetic AIs will uncover that puzzle and solve it.\nThe following is a conversation with Andrei Kapathe, previously the director of AI at\nTesla, and before that at OpenAI and Stanford.\nHe is one of the greatest scientists, engineers, and educators in the history of artificial\nintelligence.\nThis is the Lex Friedman podcast.\nTo support it, please check out our sponsors.\nAnd now, dear friends, here's Andrei Kapathe.\nWhat is a neural network?\nAnd why does it seem to do such a surprisingly good job of learning?\nWhat is a neural network?\nIt's a mathematical abstraction of the brain.\nI would say that's how it was originally developed.\nAt the end of th

**Splitting the document into chunks**

In [7]:
from gensim import corpora, models
from nltk.tokenize import sent_tokenize

class TopicBasedTextSplitter:
    def __init__(self, max_chars_per_chunk):
        self.max_chars_per_chunk = max_chars_per_chunk
        self.dictionary = corpora.Dictionary()
        self.lda_model = None

    def preprocess_text(self, text):
        # Tokenize text into sentences
        sentences = sent_tokenize(text)
        return [sentence.split() for sentence in sentences]

    def train_lda_model(self, corpus):
        # Create a dictionary from the text corpus
        self.dictionary = corpora.Dictionary(corpus)

        # Convert corpus into Bag of Words format
        corpus_bow = [self.dictionary.doc2bow(doc) for doc in corpus]

        # Train LDA model
        self.lda_model = models.LdaModel(corpus_bow, num_topics=3, id2word=self.dictionary)

    def split_into_chunks(self, text):
        # Preprocess text into sentences
        preprocessed_text = self.preprocess_text(text)

        # Train LDA model on preprocessed text
        self.train_lda_model(preprocessed_text)

        # Segment text based on topic coherence
        chunks = []
        current_chunk = ""
        for sentence in preprocessed_text:
            bow = self.dictionary.doc2bow(sentence)
            topic_id, _ = max(self.lda_model[bow], key=lambda x: x[1])
            chunk_text = " ".join(sentence)
            if len(current_chunk) + len(chunk_text) <= self.max_chars_per_chunk:
                current_chunk += " " + chunk_text
            else:
                if len(current_chunk.strip()) > 0:
                    chunks.append(current_chunk.strip())
                current_chunk = chunk_text
        if current_chunk.strip():
            chunks.append(current_chunk.strip())

        return chunks

In [8]:
splitter = TopicBasedTextSplitter(max_chars_per_chunk=1500)
chunks = splitter.split_into_chunks(text)

# Output the chunks
for i, chunk in enumerate(chunks):
    print(f"Chunk {i+1}:")
    print(chunk)
    print("\n---\n")

Chunk 1:
some kind of a crazy quantum mechanical system that somehow gives you buffer overflow, somehow gives you a rounding error in the floating point. Synthetic intelligences are kind of like the next stage of development. And I don't know where it leads to. Like at some point, I suspect the universe is some kind of a puzzle. These synthetic AIs will uncover that puzzle and solve it. The following is a conversation with Andrei Kapathe, previously the director of AI at Tesla, and before that at OpenAI and Stanford. He is one of the greatest scientists, engineers, and educators in the history of artificial intelligence. This is the Lex Friedman podcast. To support it, please check out our sponsors. And now, dear friends, here's Andrei Kapathe. What is a neural network? And why does it seem to do such a surprisingly good job of learning? What is a neural network? It's a mathematical abstraction of the brain. I would say that's how it was originally developed. At the end of the day, it'

In [9]:
# Print the number of chunks
print(f"Number of chunks: {len(chunks)}")

Number of chunks: 150


**Generating Embeddings**

In [10]:
from langchain.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings

In [11]:
db = FAISS.from_texts(chunks, embedding=OllamaEmbeddings(model='nomic-embed-text',show_progress=True))

OllamaEmbeddings: 100%|██████████| 150/150 [14:12<00:00,  5.68s/it]


**Making a retriever**

In [12]:
import re
# Check similarity search is working
query = "Why is the transformer architecture expressive in the forward pass?"
docs = db.similarity_search(query)
text=docs[0].page_content

clean_text = re.sub(r'\n', '', text)
clean_text

OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.18s/it]


"The residual connection of the weights arranged, there's a multi-layer perceptron and they're the weights stacked and so on. Um, but basically there's a message passing scheme where nodes get to look at each other, decide what's interesting and then update each other. And, uh, so I think the, um, when you get to the details of it, I think it's a very expressive function. Uh, so it can express lots of different types of algorithms and forward pass. Not only that, but the way it's designed with the residual connections, layer normalizations, the soft max attention and everything. It's also optimizable. This is a really big deal because there's lots of computers that are powerful that you can't optimize. Um, or they're not easy to optimize using the techniques that we have, which is backpropagation and gradient and sent. These are first order methods, very simple optimizers really. And so, um, you also need it to be optimizable. Um, and then lastly, you want it to run efficiently in our 

**Connecting to a Small Language Model**

Questions to ask:

* Why is the transformer architecture expressive in the forward pass?
* What design criteria does the Transformer meet?
* Why is next word prediction an effective training objective?
* What was the World Of Bits project and why did it fail?
* Why can additional sensors be a liability in an autonomous vehicle system?

Filtering and Summarization:
* Summarize the section where the neural network is explained.
* Filter out all the sentences related to artificial intelligence (AI).
* Provide a summary of Andrei Kapathe's view on neural networks.
* Identify and list sentences that contain the word "model."
* Summarize the conversation around the emergent behaviors of neural networks.
* Extract all sentences where the phrase "neural nets" is used.
* List all rhetorical questions asked by the speakers.


Sentiment and Emotion Analysis:
* What is the overall sentiment of the conversation?
* Identify any positive sentiments expressed towards AI development.
* Analyze Andrei Kapathe’s attitude towards neural networks.
* Identify expressions of uncertainty or hesitation in the transcript.

Comparisons and Contrasts:
* Compare the way neural networks are described to how the brain is discussed.
* Find any contrasts between AI and human intelligence mentioned in the conversation.
* Complex Queries and Reasoning:
* What is the relationship between neural networks and mathematical abstractions?
* How does Andrei Kapathe compare neural networks to human brains?
* How does the conversation define "emergent behavior" in AI?

Calculation-based Queries:
* How many distinct topics can be inferred from the transcript?
* Count the total number of sentences in the transcript.
* What percentage of the transcript is dedicated to the discussion of neural networks?
* What is the average sentence length in words?
* Calculate the number of times Andrei Kapathe's name appears.
* Identify the ratio of technical terms (like neural network, floating point) to non-technical language.
* Calculate the frequency of questions compared to statements in the conversation.


In [13]:
from langchain.chains.question_answering import load_qa_chain
from langchain_community.chat_models import ChatOllama

In [14]:
!ollama list

NAME                       ID              SIZE      MODIFIED   
nomic-embed-text:latest    0a109f422b47    274 MB    3 days ago    
llama2:latest              78e26419b446    3.8 GB    3 days ago    
gemma:7b                   a72c7f4d0a15    5.0 GB    3 days ago    
mistral:latest             f974a74358d6    4.1 GB    3 days ago    


In [15]:
# models 
llama = "llama2"
mistral = "mistral"
gemma = "gemma:7b"

In [16]:
def query_model(model, questions):
    # Instantiate model
    llm = ChatOllama(model=model)
    
    # Load the QA chain
    chain = load_qa_chain(llm, chain_type="stuff")
    
    # Iterate through the list of questions
    for idx, question in enumerate(questions, start=1):
        print(f"Question {idx}: {question}")
        
        # Perform similarity search
        docs = db.similarity_search(question)
        
        # Run the QA chain
        chain_response = chain.run(input_documents=docs, question=question)
        
        # Print the answer
        print(f"Answer {idx}: {chain_response}")
        
        # Print dotted line separator
        print("---------------------" )

# List of questions
questions = [
    "Why is the transformer architecture expressive in the forward pass?",
    "What design criteria does the Transformer meet?",
    "Why is next word prediction an effective training objective?",
    "What was the World Of Bits project and why did it fail?",
    "Why can additional sensors be a liability in an autonomous vehicle system?",
    "Summarize the section where the neural network is explained.",
    "Filter out all the sentences related to artificial intelligence (AI).",
    "Provide a summary of Andrei Kapathe's view on neural networks.",
    "Identify and list sentences that contain the word 'model.'",
    "Summarize the conversation around the emergent behaviors of neural networks.",
    "Extract all sentences where the phrase 'neural nets' is used.",
    "List all rhetorical questions asked by the speakers.",
    
]


### **Model 1: LLama 2**

In [17]:
#query_model(llama,questions)

Question 1: Why is the transformer architecture expressive in the forward pass?


OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.23s/it]
c:\Users\USER\Documents\Data Science\Local_llama_sql\'local-llm'\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Answer 1: The transformer architecture is expressive in the forward pass due to its design criteria that overlap and contribute to its success. Here are some reasons why:

1. Message passing scheme: The transformer uses a message passing scheme where nodes get to look at each other, decide what's interesting, and then update each other. This allows for complex computation to be expressed in a simple way, making it very expressive.
2. Layers with residual connections: The use of layers with residual connections allows the transformer to learn much more complex functions than would be possible without them. The residual connections allow the network to learn more complex and non-linear mappings between inputs and outputs.
3. Multi-layer perceptron: The transformer uses a multi-layer perceptron, which is a powerful tool for modeling complex relationships between inputs and outputs.
4. Attention mechanism: The attention mechanism allows the network to focus on specific parts of the input w

OllamaEmbeddings: 100%|██████████| 1/1 [00:03<00:00,  3.36s/it]


Answer 2: The speaker discusses the Transformer architecture and its design criteria, highlighting several key aspects:

1. Expressiveness in the forward pass: The Transformer is capable of expressing general computation as message passing between nodes, allowing it to handle a wide range of tasks.
2. Optimizability via backpropagation and gradient descent: The Transformer's architecture enables efficient optimization using backpropagation and gradient descent techniques, making it a powerful tool for training.
3. Efficiency in hardware: The Transformer is designed to run efficiently on modern hardware, such as GPUs, by leveraging parallelism and minimizing sequential operations.
4. Resilience: The Transformer architecture has proven remarkably stable since its introduction in 2016, with minimal changes made to the original design.
5. General purpose computer: The Transformer is intended to be a general-purpose computer that can be trained on arbitrary problems, such as next work predi

...

Answer 10: In this conversation, Andrei Kapathe, a prominent AI scientist, discusses the emergent behaviors of neural networks. He explains that a neural network is a mathematical abstraction of the brain, with knobs that are trainable and modifiable. These knobs allow the neural network to learn and make predictions based on large datasets. Kapathe notes that when you have a large number of knobs together, they can exhibit surprising emergent behaviors, such as next-word prediction in a massive data set from the internet. He also highlights the transformer architecture as a recent and influential idea in deep learning, which has become a general-purpose computer that can process various sensory modalities efficiently. Additionally, he mentions that neural nets will become more data-efficient in learning new tasks, but they still require massive data sets for pre-training. Overall, the conversation emphasizes the power and complexity of neural networks in making predictions and solving

OllamaEmbeddings: 100%|██████████| 1/1 [00:03<00:00,  3.05s/it]


Answer 11: 
1. "And so I think basically just like humans, neural nets will become very data efficient at learning any other new task."
2. "A lot of people in the field, I think they just talk about the amounts of like seconds and the, you know, that a person has lived pretending that this is a WLRRSA, sort of like a zero initialization of a neural net."
3. "It's like I think Eminem once said that like if he gets annoyed by a song he's written very quickly, that means it's going to be a big hit because it's too catchy. But can you describe this idea and how you're thinking about it has evolved over the months and years since since you coined it?"
4. "Yeah, could be. I mean, I'm using phrases that are common, et cetera, but I'm remixing it into a pretty unique sentence at the end of the day. But you're right, definitely there's a ton of remixing."
5. "It's like Magnus Carlsen said, I'm rated 2,900 whatever, which is pretty decent. I think you're talking very, you're not giving enough cr

OllamaEmbeddings: 100%|██████████| 1/1 [00:03<00:00,  3.02s/it]


Answer 12: 
1. "What's the meaning of life?"
2. "Can I hack it? Can I work with it?"
3. "Is there a message for me?"
4. "Am I supposed to create a message?"
5. "Why is it here?"
6. "How do we extend [the AI's capabilities]?"
7. "What are their long-term goals?"
8. "Will they figure out how to talk shit to me?"
9. "Can they become oracles?"
---------------------


### **Model 2: Mistral**

In [18]:
#query_model(mistral,questions)

Question 1: Why is the transformer architecture expressive in the forward pass?


OllamaEmbeddings:   0%|          | 0/1 [00:00<?, ?it/s]

OllamaEmbeddings: 100%|██████████| 1/1 [00:03<00:00,  3.17s/it]


Answer 1:  The Transformer architecture is expressive in the forward pass due to its design as a message passing scheme, where nodes (also known as attention heads) get to look at each other and communicate with one another. Each node stores a vector, and they can broadcast information about what they are looking for (keys) and what information they have (values). This allows the model to learn complex patterns and relationships between different pieces of data, making it versatile for a wide range of tasks. The Transformer's ability to express general computation in this way is one of the key reasons for its success and popularity. Additionally, the use of attention mechanisms enables the model to selectively focus on important information and ignore irrelevant details, further increasing its expressiveness. Overall, the Transformer architecture's message passing scheme and attention mechanisms make it a powerful and flexible tool for solving a variety of machine learning tasks.
-----

OllamaEmbeddings: 100%|██████████| 1/1 [00:03<00:00,  3.72s/it]


...

Answer 10:  The conversation revolves around the surprising and emergent behaviors exhibited by neural networks. Andrei Kapathe explains that neural networks are mathematical abstractions of the brain, consisting of a sequence of matrix multipliers (dot products) with non-linearity and trainable knobs. These knobs need to be properly set for the neural net to perform desired tasks like classifying images. The network's surprising abilities become more evident when it handles complex problems, such as next word prediction in massive datasets from the internet. Kapathe also mentions the transformer architecture, which has gained popularity due to its ability to process various sensory modalities and can learn new tasks with minimal examples, resembling a general-purpose computer. He suggests that humans might have a similar passive, self-supervised learning mechanism running in the background during their lifetime, helping them learn efficiently over time without conscious effort.
------

OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.86s/it]


Answer 11: 1. "So a lot of these GPTs, you can do tasks like sentiment analysis or translation or so on just by being prompted with very few examples. Here's the kind of thing I want you to do. Here's an input sentence, here's the translation into German. Input sentence, translation to German. Input sentence blank, and the neural net will complete the translation to German just by looking at sort of the example you've provided. And so that's an example of a very few shot learning in the activations of the neural net instead of the weights of the neural net."

2. "A lot of people in the field, I think they just talk about the amounts of like seconds and the, you know, that a person has lived pretending that this is a WLRRSA, sort of like a zero initialization of a neural net."

3. "It's like I think Eminem once said that like if he gets annoyed by a song he's written very quickly, that means it's going to be a big hit because it's too catchy. But can you describe this idea and how you'r

OllamaEmbeddings: 100%|██████████| 1/1 [00:03<00:00,  3.04s/it]


Answer 12: 1. Anyone can choose their own meaning of life because we are a conscious entity and it's beautiful. Number one. (Question about personal philosophy of life)
2. What the hell is all this and like, why? And if you look at the inter fundamental physics and the quantum field theory and the standard model, they're like very complicated. And there's this like 19 free parameters of our universe and like, what's going on with all this stuff and why is it here? (Question about the nature of the universe)
3. Can I hack it? Can I work with it? Is there a message for me? Am I supposed to create a message? (Questions about the purpose of existence)
4. For that, that's going to be take a very long time. So the why question boils down from an engineering perspective to how do we extend? (Question about extending human life and answering the ultimate question)
5. Did you hear it? Like they'll do gossip. They'll do, uh, they'll try to plant seeds of suspicion to other humans that you love a

### **Model 3: Gemma**

In [22]:
query_model(gemma,questions)

Question 1: Why is the transformer architecture expressive in the forward pass?


OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.99s/it]


ValueError: Ollama call failed with status code 500. Details: {"error":"llama runner process has terminated: error:failed to create context with model 'C:\\Users\\USER\\.ollama\\models\\blobs\\sha256-ef311de6af9db043d51ca4b1e766c28e0a1ac41d60420fed5e001dc470c64b77'"}

"Compare the way neural networks are described to how the brain is discussed.",
    "How many distinct topics can be inferred from the transcript?",
    "Count the total number of sentences in the transcript.",
    "What percentage of the transcript is dedicated to the discussion of neural networks?",
    "What is the average sentence length in words?",
    "Calculate the number of times Andrei Kapathe's name appears.",
    "Identify the ratio of technical terms (like neural network, floating point) to non-technical language.",
    "Calculate the frequency of questions compared to statements in the conversation.",
    "Find any contrasts between AI and human intelligence mentioned in the conversation.",
    "What is the relationship between neural networks and mathematical abstractions?",
    "How does Andrei Kapathe compare neural networks to human brains?",
    "How does the conversation define 'emergent behavior' in AI?",
    "What is the overall sentiment of the conversation?",
    "Identify any positive sentiments expressed towards AI development.",
    "Analyze Andrei Kapathe’s attitude towards neural networks."
    "Identify expressions of uncertainty or hesitation in the transcript."